# AnkiConnect + OpenAI TTS + 예문 자동 업데이트

원본 덱을 복사해 `*_new` 덱을 만든 뒤, **새 덱만** 업데이트합니다.

먼저 로컬 폴더(`./anki_handler/audio`, `./anki_handler/sentence`)에 자산을 생성한 뒤, 그 파일들을 로드하여 노트를 업데이트합니다.

In [103]:
import os
import importlib.util
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

# anki_handler.py를 직접 로드 (동일 이름 패키지 충돌 방지)
spec = importlib.util.spec_from_file_location(
    "anki_handler_script", Path("anki_handler.py")
)
ah = importlib.util.module_from_spec(spec)
spec.loader.exec_module(ah)

SOURCE_DECK = os.getenv("ANKI_DECK", "hackers_toefl")
TARGET_DECK = os.getenv("ANKI_TARGET_DECK", f"{SOURCE_DECK}_new")
ANKI_URL = os.getenv("ANKI_CONNECT_URL", "http://localhost:8760")
FRONT_FIELD = os.getenv("ANKI_FRONT_FIELD", "Front")
BACK_FIELD = os.getenv("ANKI_BACK_FIELD", "Back")

AUDIO_DIR = Path(os.getenv("ANKI_AUDIO_DIR", "anki_handler/audio"))
SENTENCE_DIR = Path(os.getenv("ANKI_SENTENCE_DIR", "anki_handler/sentence"))

TTS_MODEL = os.getenv("OPENAI_TTS_MODEL", "gpt-4o-mini-tts")
TTS_VOICE = os.getenv("OPENAI_TTS_VOICE", "coral")
TTS_INSTRUCTIONS = os.getenv(
    "OPENAI_TTS_INSTRUCTIONS",
    "Speak in a natural British English accent and pronounce the word clearly.",
)
TEXT_MODEL = os.getenv("OPENAI_TEXT_MODEL", "gpt-4.1-mini")
NUM_SENTENCES = int(os.getenv("ANKI_NUM_SENTENCES", "2"))
raw_limit = os.getenv("ANKI_LIMIT", "10")
LIMIT = None if raw_limit.strip() in ("", "0") else int(raw_limit)

ah.require_api_key()
ah.ensure_dirs(AUDIO_DIR, SENTENCE_DIR)

In [104]:
# 1) 원본 덱에서 노트 가져오기
source_notes = ah.fetch_notes(ANKI_URL, SOURCE_DECK)
len(source_notes)

1811

In [105]:
# 2) 새 덱 준비 (원본 덱 복사본 생성)
target_notes = ah.prepare_target_notes(ANKI_URL, source_notes, TARGET_DECK, limit=LIMIT)
len(source_notes), len(target_notes)

(1811, 1811)

In [106]:
# 3) 샘플 노트 확인 (새 덱)
note_id, fields = next(iter(target_notes.items()))
fields.get(FRONT_FIELD, ""), fields.get(BACK_FIELD, "")[:200]

('exploit',
 'utilize, use, make use of, take advantage of (부당하게) 이용하다<br><br>[sound:anki_uk_exploit.mp3]<br>Example: The company tried to exploit new markets abroad.<br>She learned how to exploit her talents effec')

In [107]:
# 4) 자산 생성/로딩 헬퍼
def generate_assets_for_note(
    note_id, fields, overwrite_audio=False, overwrite_sentence=False
):
    return ah.generate_assets(
        note_id,
        fields,
        FRONT_FIELD,
        AUDIO_DIR,
        SENTENCE_DIR,
        TTS_MODEL,
        TTS_VOICE,
        TTS_INSTRUCTIONS,
        TEXT_MODEL,
        NUM_SENTENCES,
        overwrite_audio,
        overwrite_sentence,
    )


def update_note_from_assets(note_id, fields, overwrite_audio=False):
    front = fields.get(FRONT_FIELD, "").strip()
    back = fields.get(BACK_FIELD, "")
    if not front:
        raise ValueError("Front is empty")

    audio_path, sentence_path = ah.resolve_asset_paths(
        note_id, front, AUDIO_DIR, SENTENCE_DIR
    )
    audio_filename = audio_path.name

    audio_bytes, sentence = ah.load_assets(audio_path, sentence_path)
    audio_tag = ah.store_audio_in_anki(
        ANKI_URL, audio_filename, audio_bytes, overwrite_audio
    )

    new_back = ah.build_new_back(back, audio_tag, sentence)
    return front, new_back

In [108]:
# 5) 단일 노트 자산 생성
generate_assets_for_note(
    note_id, fields, overwrite_audio=False, overwrite_sentence=False
)

('exploit',
 PosixPath('anki_handler/audio/anki_uk_exploit.mp3'),
 PosixPath('anki_handler/sentence/anki_sentence_exploit.txt'))

In [109]:
# 6) 단일 노트 업데이트 준비(로컬 자산 로드)
front, new_back = update_note_from_assets(note_id, fields, overwrite_audio=False)
front, new_back[:300]

('exploit',
 'utilize, use, make use of, take advantage of (부당하게) 이용하다<br><br>[sound:anki_uk_exploit.mp3]<br>Example: The company tried to exploit new markets abroad.<br>She learned how to exploit her talents effectively.<br>They were careful not to exploit the situation unfairly.')

In [110]:
# 7) 단일 노트 업데이트 (실행 시 실제 반영)
ah.update_note_fields(
    ANKI_URL,
    note_id,
    {
        FRONT_FIELD: front,
        BACK_FIELD: new_back,
    },
)

In [111]:
# 8) 배치 처리 (1) 자산 생성 -> (2) 자산 로드 & 업데이트
DRY_RUN = False
OVERWRITE_AUDIO = False
OVERWRITE_SENTENCE = False

items = list(target_notes.items())
if LIMIT is not None:
    items = items[:LIMIT]

# (1) 자산 생성
for note_id, fields in items:
    try:
        generate_assets_for_note(
            note_id,
            fields,
            overwrite_audio=OVERWRITE_AUDIO,
            overwrite_sentence=OVERWRITE_SENTENCE,
        )
    except Exception as exc:
        print(f"[WARN] generate note={note_id} error={exc}")

processed = 0
skipped = 0

# (2) 자산 로드 & 업데이트
for note_id, fields in items:
    try:
        front, new_back = update_note_from_assets(
            note_id, fields, overwrite_audio=OVERWRITE_AUDIO
        )

        if DRY_RUN:
            print(f"[DRY RUN] note={note_id} word={front}")
        else:
            ah.update_note_fields(
                ANKI_URL,
                note_id,
                {
                    FRONT_FIELD: front,
                    BACK_FIELD: new_back,
                },
            )
        processed += 1
    except Exception as exc:
        print(f"[WARN] update note={note_id} error={exc}")
        skipped += 1

print(f"Done. processed={processed}, skipped={skipped}")

KeyboardInterrupt: 